In [1]:
#log into kaggle to access dataset
import os
from collections import defaultdict
import kagglehub
from tqdm import tqdm

c:\Users\Marcu\AppData\Local\Programs\Python\Python311\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
#please ikke offentliggør min api key. 
os.environ['KAGGLE_USERNAME'] = "marcusgaleajacobsen"
os.environ['KAGGLE_KEY'] = "32a3003f52c97053841ea46c492128dc"

# Download data

In [3]:
# Download latest version
datapath = kagglehub.dataset_download("netflix-inc/netflix-prize-data")
os.listdir(datapath)

['combined_data_1.txt',
 'combined_data_2.txt',
 'combined_data_3.txt',
 'combined_data_4.txt',
 'movie_titles.csv',
 'probe.txt',
 'qualifying.txt',
 'README']

# Custom Movie and User data structures
One data structure suggestion is to store the users and movies in their own dictionaries with the user_id and movie_id as the keys. The advantage of dictionaries is that the access time is $O(1)$ for both movies and users which is very fast. 
```python
users = {
    '123': User('123')
    ...
}
movies = {
    '1': Movie(1, 'Toy Story', '1995')
    ...
}
```

In [4]:
from src.structures import Movie, User
from src.data_methods import read_movies,read_viewers, dict_to_df, read_df
movies = read_movies(datapath)
users = read_viewers(datapath, movies, datafiles = ['combined_data_1.txt'], with_tqdm= True, n_lines = 100000) #read only 100000 reviews for now

100%|██████████| 100000/100000 [00:01<00:00, 65830.68it/s]


In [5]:
movies

defaultdict(src.structures.Movie,
            {'1': Movie(1, Dinosaur Planet, 2003) seen by 547 users,
             '2': Movie(2, Isle of Man TT 2004 Review, 2004) seen by 145 users,
             '3': Movie(3, Character, 1997) seen by 2012 users,
             '4': Movie(4, Paula Abdul's Get Up & Dance, 1994) seen by 142 users,
             '5': Movie(5, The Rise and Fall of ECW, 2004) seen by 1140 users,
             '6': Movie(6, Sick, 1997) seen by 1019 users,
             '7': Movie(7, 8 Man, 1992) seen by 93 users,
             '8': Movie(8, What the #$*! Do We Know!?, 2004) seen by 14910 users,
             '9': Movie(9, Class of Nuke 'Em High 2, 1991) seen by 95 users,
             '10': Movie(10, Fighter, 2001) seen by 249 users,
             '11': Movie(11, Full Frame: Documentary Shorts, 1999) seen by 198 users,
             '12': Movie(12, My Favorite Brunette, 1947) seen by 546 users,
             '13': Movie(13, Lord of the Rings: The Return of the King: Extended Edition: B

In [6]:
users

defaultdict(src.structures.User,
            {'1488844': User 1488844 has rated 4 movies,
             '822109': User 822109 has rated 1 movies,
             '885013': User 885013 has rated 2 movies,
             '30878': User 30878 has rated 5 movies,
             '823519': User 823519 has rated 5 movies,
             '893988': User 893988 has rated 2 movies,
             '124105': User 124105 has rated 1 movies,
             '1248029': User 1248029 has rated 3 movies,
             '1842128': User 1842128 has rated 2 movies,
             '2238063': User 2238063 has rated 1 movies,
             '1503895': User 1503895 has rated 1 movies,
             '2207774': User 2207774 has rated 1 movies,
             '2590061': User 2590061 has rated 2 movies,
             '2442': User 2442 has rated 2 movies,
             '543865': User 543865 has rated 2 movies,
             '1209119': User 1209119 has rated 3 movies,
             '804919': User 804919 has rated 1 movies,
             '1086807'

# Methods

In [7]:
#get ratings for a user
user = users['1488844']
print(user)
user_ratings = user.get_ratings()
print("ratings on the format movie_id : rating ",user.ratings)

User 1488844 has rated 4 movies
ratings on the format movie_id : rating  defaultdict(<class 'float'>, {'1': 3, '8': 4, '17': 2, '30': 3})


In [8]:
#get ratings for a movie
movie = movies['1']
print(movie)
movie_ratings = movie.get_ratings()
print("ratings: ", movie_ratings)


Dinosaur Planet, 2003
ratings:  [3, 5, 4, 4, 3, 3, 4, 3, 4, 3, 4, 5, 3, 3, 4, 4, 4, 3, 4, 5, 3, 3, 4, 4, 3, 4, 4, 5, 4, 5, 5, 3, 4, 4, 5, 3, 2, 4, 4, 2, 1, 2, 4, 3, 3, 5, 5, 4, 5, 3, 1, 4, 3, 3, 4, 2, 3, 4, 3, 3, 4, 5, 4, 5, 1, 5, 2, 4, 5, 4, 4, 2, 4, 4, 3, 4, 5, 5, 5, 5, 1, 3, 4, 3, 2, 1, 5, 3, 4, 2, 4, 4, 4, 1, 4, 3, 3, 4, 4, 5, 3, 1, 5, 5, 4, 3, 5, 2, 3, 5, 5, 5, 5, 5, 4, 3, 5, 4, 2, 3, 4, 1, 5, 4, 4, 5, 4, 3, 5, 3, 5, 5, 1, 3, 4, 5, 5, 2, 4, 3, 3, 5, 4, 5, 5, 4, 3, 1, 3, 4, 3, 2, 3, 4, 1, 4, 4, 5, 3, 2, 2, 4, 3, 2, 3, 4, 5, 4, 4, 3, 3, 1, 3, 4, 4, 2, 3, 3, 4, 3, 4, 4, 2, 4, 4, 4, 4, 3, 4, 4, 3, 5, 5, 5, 4, 5, 5, 4, 4, 4, 4, 5, 5, 5, 3, 3, 2, 4, 5, 5, 2, 1, 4, 3, 5, 4, 5, 5, 5, 4, 4, 4, 4, 3, 3, 5, 5, 5, 4, 5, 3, 3, 3, 5, 4, 4, 5, 5, 1, 4, 3, 5, 5, 4, 3, 4, 4, 5, 4, 3, 3, 1, 4, 3, 4, 4, 4, 4, 4, 5, 4, 5, 4, 4, 2, 5, 1, 4, 5, 4, 4, 5, 3, 5, 5, 3, 4, 4, 5, 3, 4, 4, 3, 3, 3, 4, 4, 2, 5, 4, 5, 5, 3, 4, 4, 5, 4, 3, 5, 5, 1, 5, 5, 5, 5, 4, 4, 3, 5, 2, 4, 4, 5, 5, 3, 3, 3, 5, 4, 4, 4, 3, 5

In [9]:
# # Normalize ratings for each user to have mean 0 and variance 1
# for user in users.values():
#     user.normalize_ratings()

# Alternative data structure: DataFrame
Another suggestion is to use a pandas DataFrame to store the data. The advantage of using a DataFrame is that it is very fast and has a lot of built-in functionality. (It's also more memory efficient than dictionaries). The downside is that access time for users/movies is $O(n)$ which is slower than a dictionary (where $n$ is the number of rows in the DataFrame).

In [14]:
df = read_df(datapath, datafiles=['combined_data_1.txt'], n_lines=100000)
df

,user_id,movie_id,rating
0,1488844,1,3
1,822109,1,5
2,885013,1,4
3,30878,1,4
4,823519,1,3
...,...,...,...
99965,735848,30,4
99966,254710,30,4
99967,865725,30,4
99968,568153,30,3


In [16]:
#all ratings for user 1488844
df[df.user_id == "1488844"]

,user_id,movie_id,rating
0,1488844,1,3
5149,1488844,8,4
24352,1488844,17,2
93266,1488844,30,3


In [17]:
#all ratings for movie 1
df[df.movie_id == "1"]

,user_id,movie_id,rating
0,1488844,1,3
1,822109,1,5
2,885013,1,4
3,30878,1,4
4,823519,1,3
...,...,...,...
542,1790158,1,4
543,1403184,1,3
544,1535440,1,4
545,1426604,1,4
